# Mphasis HyperGraf Medical Appointment No-Show Predictor

Ensemble Machine Learning algorithm-based solution which predicts the probability of patient no-shows for outpatient medical appointments. Patient no-shows for scheduled appointments is a significant challenge for healthcare providers. Patient no-shows leads to inefficient utilization of scarce healthcare resources and service quality risks. This solution predicts the likelihood of patient no-shows, based on patients’ appointment and medical information. The solution assists health care providers to make informed patient scheduling and reminder decisions.  

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Mphasis HyperGraf Medical Appointment No-Show Predictor using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis HyperGraf Medical Appointment No-Show Predictor. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Mphasis HyperGraf Medical Appointment No-Show Predictor
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/medical-noshow-v1'

In [2]:
import pandas as pd
import numpy as np
import json
import os
import boto3
from zipfile import ZipFile
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image, display

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='mednoshow'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [ ]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

#### Instructions

##### Supported content types: 
csv file only  

##### Mandatory fields: 
Patient_Id, Gender, Age,  Schedule_Weekday, Appointment_Weekday, Lead_Time, Prior_Appointments, Prior_No_Show_Percent, Appointment_Notification, Beneficiary_Govt_Medical_Scheme, Hypertension, Diabetes, Alcoholism, Differently Abled.  

##### Input field descriptions: 

1) Patient_Id: Numeric or alpha-numeric value to uniquely identify a patient. For example, ‘P01’. 

2) Gender: ‘Male’ or ‘Female’. 

3) Age: Numeric value. For example, a child of 10 months has age ‘0’.  

4) Schedule_Weekday: Numeric value for the day on which appointment is confirmed.  For example, ‘1’: Monday, ‘2’: Tuesday, ’3’: Wednesday, ’4’: Thursday, ‘5’: Friday, ’6’: Saturday, ‘7’: Sunday. 

5) Appointment_Weekday: Numeric value for the day on which appointment is scheduled. For example, ‘1’: Monday, ‘2’: Tuesday,’3’: Wednesday,’4’: Thursday, ‘5’: Friday,’6’: Saturday, ‘7’: Sunday  

6) Prior_Appointments: Number of prior appointments for the patient.  

7) Lead_Time: Number of days between the date of appointment booking/confirmation and the date of scheduled appointment. For example, If confirmation date is July 1 and appointment date is July 6, then Lead_Time value is ‘5’. 

8) Prior_No_Show_Percent: This is the ratio of prior no-shows to the total appointments. For example, if a patient did not show up for 10 appointments from a total of 100 appointments, the Prior_No_Show_Percent is ‘0.1’.  

9) Appointment_Notification: If the patient has received an appointment confirmation notification, the value is ‘Y’ Else, the value is ‘N’  

10) Beneficiary_Govt_Medical_Scheme: If the patient is a beneficiary of any Government sponsored healthcare scheme, the value is ‘Y’ Else, the value is ‘N’.  

11) Hypertension: If a patient has a medical history of hypertension, the value is ‘Y’ Else, the value is ‘N’.  

12) Diabetes - If a patient has a medical history of diabetes, the value is ‘Y’ Else, the value is ‘N’. 

13) Alcoholism - If a patient has a history of alcohol consumption, the value is ‘Y’ Else, the value is ‘N’.  

14) Differently_Abled – If a patient is differently abled, the value is ‘Y’, Else the value is ‘N’. 

In [7]:
file_name = 'sample_input.csv'

#### C. Perform real-time inference

In [8]:
pd.read_csv('sample_input.csv')

,Patient_Id,Gender,Age,Beneficiary_Govt_Medical_Scheme,Hypertension,Diabetes,Alcoholism,Differently_Abled,Appointment_Notification,Lead_Time,Appointment_Weekday,Schedule_Weekday,Prior_Appointments,Prior_No_Show_Percent
0,1,Male,62,N,Y,Y,N,Y,Y,26,1,3,0,0.0
1,2,Male,52,N,N,N,N,N,Y,16,3,1,0,0.0
2,3,Female,33,N,N,N,N,N,N,0,1,1,1,1.0
3,4,Female,30,N,N,N,N,N,N,0,2,2,0,0.0
4,5,Male,4,N,N,N,N,N,Y,27,1,2,0,0.0
5,6,Male,56,N,N,N,N,N,Y,14,4,4,0,0.0
6,7,Female,58,Y,Y,Y,N,N,N,0,2,2,2,0.5
7,8,Male,50,N,Y,N,N,Y,N,28,3,3,0,0.0


In [9]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 output.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- Supported content types: 'csv' files only. 

- Output file columns: Patient_Id & No_Show_Probability. 

- The No_Show_Probability corresponding to a Patient_Id  is the probability of patient not 	                    showing up for the appointment.

In [10]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df.head()

Sample Output:


,Patient_Id,No_Show_Probability
0,1,0.234
1,2,0.451
2,3,0.966
3,4,0.084
4,5,0.614


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [11]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [12]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/mednoshow


In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [14]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output2.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/sample_input.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [15]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output2.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df.head()

Sample Output:


,Patient_Id,No_Show_Probability
0,1,0.234
1,2,0.451
2,3,0.966
3,4,0.084
4,5,0.614


### 4. Clean-up

#### A. Delete the model

In [16]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.